In [1]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [3]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 967 images belonging to 12 classes.
Found 428 images belonging to 14 classes.


In [5]:
# Define the VGG16 base model
def create_InceptionResNetV2_model():
    InceptionResNetV2_model = InceptionResNetV2(
        include_top=False, 
        input_shape=(img_height, img_width, 3)
        )
    InceptionResNetV2_model.trainable = False

    model = Sequential([
        InceptionResNetV2_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(12, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [6]:
#MODEL fit
model = create_InceptionResNetV2_model()
model.fit(train_generator, epochs=10, validation_data=test_generator)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                98368     
                                                                 
 batch_normalization_203 (Ba  (None, 64)               256       
 tchNormalization)                                               
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "d:\study\codes\engineering_codes\research\conda_env\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\study\codes\engineering_codes\research\conda_env\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 205

In [ ]:
#predict
predictions = model.predict(test_generator)



7/7 [==============================] - 10s 115ms/step


In [ ]:
#saving predictions in csv file
np.savetxt("../model_saved_files/predictions/InceptionResNetV2_predictions.csv", predictions, delimiter=",")

# Save the model
# model.save("../model_saved_files/models_weights/InceptionResNetV2.h5")